In [6]:
import os
import shutil

from arc.common import almost_equal_lists, read_yaml_file

from t3 import T3
from tests.common import almost_equal, run_minimal
from t3.simulate.cantera_IDT import CanteraIDT, get_t_and_p_lists, get_idt_per_phi_p_condition, plot_idt_vs_temperature
from t3.utils.fix_cantera import fix_cantera

In [7]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
project_name = "2BF_xmr_2038"
iteration_num = 2

SIM_DIR = os.path.join(os.getcwd(),f'data/{project_name}')
FIG_DIR_IDT = os.path.join(SIM_DIR,f'iteration_{iteration_num}/Figures/')

In [9]:
#Load T3 input file of your project into a dictionairy
yaml_file_path = os.path.join(SIM_DIR,'input.yml')
input_dict = read_yaml_file(yaml_file_path)

#print the T3 input file dictionairy:
for k,v in input_dict.items():
    print(k,v)

#create a t3 object from your T3 run:
rmg_args = input_dict["rmg"]
t3_args = input_dict["t3"]
qm_args = input_dict["qm"]

t3 = T3(project=project_name,
        t3=t3_args,
        rmg=rmg_args,
        qm=qm_args,
        )

t3.iteration = iteration_num

project 2BF
t3 {'options': {'library_name': '2BF_xmr_2038', 'max_T3_iterations': 15, 'max_RMG_exceptions_allowed': 10, 'modify_concentration_ranges_in_reverse': False, 'modify_concentration_ranges_together': True, 'num_sa_per_temperature_range': 3, 'num_sa_per_pressure_range': 3, 'num_sa_per_concentration_range': 3}, 'sensitivity': {'adapter': 'RMGConstantTP', 'SA_threshold': 0.01, 'pdep_SA_threshold': 0.001, 'ME_methods': ['CSE', 'MSC'], 'top_SA_species': 10, 'top_SA_reactions': 10}}
rmg {'database': {'thermo_libraries': ['primaryThermoLibrary', 'BurkeH2O2', '2BF_thermo', '2BF_thermo_wo_rotors', '2FFOH_thermo', '2FFOH_thermo_wo_rotors', '/home/nellymitnik/runs/T3/2BF/xmr_2033/2BF_xmr_2027_b_lib.py', '/home/nellymitnik/runs/T3/2BF/xmr_2033/2BF_xmr_2029_lib.py', 'thermo_DFT_CCSDTF12_BAC', 'DFT_QCI_thermo', 'Spiekermann_refining_elementary_reactions', 'CurranPentane', 'C3', 'CBS_QB3_1dHR', 'FFCM1(-)', 'JetSurF2.0', 's3_5_7_ane', 'naphthalene_H', 'USC-Mech-ii', 'heavy_oil_ccsdtf12_1dHR', 

In [10]:
t3.project_directory = SIM_DIR
t3.set_paths()
t3.rmg['reactors'] = [{'type': 'gas batch constant T P',
                        'T': [700, 1990], 'P': [1, 50],
                        'termination_rate_ratio': 0.01},
                        ]
t3.rmg['species'] = [{'label': '2BF', 'smiles': 'CCCCC1=CC=CO1', 'concentration': 0, 'role': 'fuel',
                        'equivalence_ratios': [0.5, 1.5]},
                        {'label': 'O2', 'smiles': '[O][O]', 'concentration': 0, 'role': 'oxygen'},
                        {'label': 'N2', 'smiles': 'N#N', 'concentration': 0, 'role': 'nitrogen'}]
ct_adapter = CanteraIDT(t3=t3.t3,
                        rmg=t3.rmg,
                        paths=t3.paths,
                        logger=t3.logger,
                        atol=t3.rmg['model']['atol'],
                        rtol=t3.rmg['model']['rtol'],
                        )
ct_adapter.simulate()
plot_idt_vs_temperature(idt_dict=ct_adapter.idt_dict,figs_path=FIG_DIR_IDT)

radical label: OH(18)
Running a simulation using CanteraIDT...
